In [1]:
import pyspark
from delta import *

In [3]:
builder = (pyspark.sql.SparkSession.builder.appName("Schema_Evolution")
           .config("spark.sql.extensions","io.delta.sql.DeltaSparkSessionExtension")
           .config("spark.sql.catalog.spark_catalog","org.apache.spark.sql.delta.catalog.DeltaCatalog")
         )

In [5]:
spark=configure_spark_with_delta_pip(builder).getOrCreate()

:: loading settings :: url = jar:file:/Users/vk/Library/Python/3.9/lib/python/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/vk/.ivy2/cache
The jars for the packages stored in: /Users/vk/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-0d8a3e86-a242-411f-9736-0d488ef00100;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.2.0 in central
	found io.delta#delta-storage;2.2.0 in central
	found org.antlr#antlr4-runtime;4.8 in central
:: resolution report :: resolve 89ms :: artifacts dl 2ms
	:: modules in use:
	io.delta#delta-core_2.12;2.2.0 from central in [default]
	io.delta#delta-storage;2.2.0 from central in [default]
	org.antlr#antlr4-runtime;4.8 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0   |   0   |   

23/04/23 12:50:51 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


# Create Delta Table

In [6]:
df = spark.createDataFrame([('ABC',24),('WSX',25),('EDC',27)]).toDF("Name","Age")

In [7]:
df.show()

+----+---+
|Name|Age|
+----+---+
| ABC| 24|
| WSX| 25|
| EDC| 27|
+----+---+



In [10]:
df.write.format("delta").saveAsTable("Schema_Evolution_Test1")

In [11]:
DeltaTable.isDeltaTable(spark,"spark-warehouse/schema_evolution_test1/")

True

In [16]:
spark.sql("DESCRIBE DETAIL schema_evolution_test1").select("format","name","location").show(truncate=False)

+------+------------------------------+-------------------------------------------------------------------------+
|format|name                          |location                                                                 |
+------+------------------------------+-------------------------------------------------------------------------+
|delta |default.schema_evolution_test1|file:/Users/vk/Documents/Deltalake/spark-warehouse/schema_evolution_test1|
+------+------------------------------+-------------------------------------------------------------------------+



# Append new records with schema change

In [17]:
df_1 = spark.createDataFrame([("RFV",35,"USA"),("TGB",36,"SG")]).toDF("Name","Age","Country")
df_1.show()

+----+---+-------+
|Name|Age|Country|
+----+---+-------+
| RFV| 35|    USA|
| TGB| 36|     SG|
+----+---+-------+



In [19]:
df_1.write.format("delta").mode("append").saveAsTable("schema_evolution_test1")

AnalysisException: A schema mismatch detected when writing to the Delta table (Table ID: babd4b83-b970-46c0-a32c-bbbdf013308d).
To enable schema migration using DataFrameWriter or DataStreamWriter, please set:
'.option("mergeSchema", "true")'.
For other operations, set the session configuration
spark.databricks.delta.schema.autoMerge.enabled to "true". See the documentation
specific to the operation for details.

Table schema:
root
-- Name: string (nullable = true)
-- Age: long (nullable = true)


Data schema:
root
-- Name: string (nullable = true)
-- Age: long (nullable = true)
-- Country: string (nullable = true)

         

# Enable Schema Evolution with mergeSchema Set to TRUE

In [27]:
df_1.write.option("mergeSchema","True").mode("append").format("delta").saveAsTable("Schema_Evolution_Test1")

In [28]:
spark.sql("SELECT * FROM Schema_Evolution_Test1").show()

+----+---+-------+
|Name|Age|Country|
+----+---+-------+
| RFV| 35|    USA|
| TGB| 36|     SG|
| EDC| 27|   null|
| WSX| 25|   null|
| ABC| 24|   null|
+----+---+-------+



In [31]:
spark.sql("DESCRIBE HISTORY Schema_Evolution_Test1").select("version","operation","operationParameters").show(truncate=False)

+-------+----------------------+-----------------------------------------------------------------------------+
|version|operation             |operationParameters                                                          |
+-------+----------------------+-----------------------------------------------------------------------------+
|1      |WRITE                 |{mode -> Append, partitionBy -> []}                                          |
|0      |CREATE TABLE AS SELECT|{isManaged -> true, description -> null, partitionBy -> [], properties -> {}}|
+-------+----------------------+-----------------------------------------------------------------------------+



# Enable autoMerge 

In [32]:
spark.conf.set("spark.databricks.delta.schema.autoMerge.enabled","true")

In [34]:
df = spark.createDataFrame([("YHN",),("UJM",)]).toDF("Name")

In [35]:
df.write.format("delta").mode("append").saveAsTable("Schema_Evolution_Test1")

In [36]:
spark.sql("SELECT * FROM Schema_Evolution_Test1").show()

+----+----+-------+
|Name| Age|Country|
+----+----+-------+
| RFV|  35|    USA|
| TGB|  36|     SG|
| EDC|  27|   null|
| WSX|  25|   null|
| ABC|  24|   null|
| YHN|null|   null|
| UJM|null|   null|
+----+----+-------+



In [37]:
spark.sql("DESCRIBE HISTORY Schema_Evolution_Test1").select("version","operation","operationParameters").show(truncate=False)

+-------+----------------------+-----------------------------------------------------------------------------+
|version|operation             |operationParameters                                                          |
+-------+----------------------+-----------------------------------------------------------------------------+
|2      |WRITE                 |{mode -> Append, partitionBy -> []}                                          |
|1      |WRITE                 |{mode -> Append, partitionBy -> []}                                          |
|0      |CREATE TABLE AS SELECT|{isManaged -> true, description -> null, partitionBy -> [], properties -> {}}|
+-------+----------------------+-----------------------------------------------------------------------------+

